In this notebook we look at the language model class and the lightning wrapper

In [1]:
#First we fix the relative imports
import os
import sys
# M
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from modules.LanguageModels.LstmLanguageModel import LSTMLanguageModel

In [3]:
## Every language model should implement the interface defined in the base language model

#We first look at a small example lstm based language model

from daily_dialog.DialogTokenizer import get_daily_dialog_tokenizer
my_tokenizer = get_daily_dialog_tokenizer(tokenizer_location='../daily_dialog/tokenizer.json', )


#This imports a fresh language model
language_model = LSTMLanguageModel(my_tokenizer.get_vocab_size())


In [4]:
#We can give a context and ask to complete it
from utils.utils import encode
context = "Hi how are you today? [SEP]"
ids = encode(my_tokenizer, context)
print(ids)
completed_dialogue = language_model.complete_dialogue(ids)
print(completed_dialogue)


tensor([427, 219, 178, 127, 484,  33,   1])
tensor([ 427,  219,  178,  127,  484,   33,    1,  549,  817,  430,  817,  150,
        1020, 1402,  817,  994, 1183, 2028, 1698,  817,  636,  150, 1020, 1468,
        1053,  673,  994,  430, 2028,  994, 2044, 1323, 1402,  929,  549,  102,
        1165,  121,  549,  549, 1610,  712, 1610, 1402,  517,  712,  517, 1053,
         871,  202,  817, 1402,  929,  430,  549,  509, 1183,  666,  827, 1183,
        2028, 1402, 1721, 1020,  430,  549,  150,  509,  666,  549,  871,  871,
        1364, 1183, 1721,  858, 1835, 1835,   79, 1183, 1835, 1183, 1020, 1020,
          79,   79,  693, 1183,  549, 1721,  430, 1721,  929, 1835, 1183, 1835,
         191,  858,  673,  994])


In [5]:
## we decode the completed dialogue to see its response, which mostly nonsense as it is not trained yet
from utils.utils import decode
decoded_completed_dialogue = decode(my_tokenizer, completed_dialogue)
print(decoded_completed_dialogue)

hi how are you today ? [SEP] ke price lot priceed prefer beijing price seemswh quiet fact pricetinged prefercially probably el seems lot quiet seems div wonderful beijing understand ke4 weather se ke ke popular around popular beijingty aroundty probably enoughim price beijing understand lot kebewhqu pretwh quiet beijing system prefer lot keedbequ ke enough enoughvedwh systemember soft softcwh softwh prefer preferccousewh ke system lot system understand softwh softionember el seems


In [6]:
### Loading a trained model, which you can see already gives a better result, it is locally consistent.
location = "../models/small_lm_test.pt"
trained_language_model = LSTMLanguageModel.load(location) 

In [7]:
ids = encode(my_tokenizer, context)
print(ids)
completed_dialogue = trained_language_model.complete_dialogue(ids)
print(completed_dialogue)
decoded_completed_dialogue = decode(my_tokenizer, completed_dialogue)
print(decoded_completed_dialogue)

tensor([427, 219, 178, 127, 484,  33,   1])
tensor([ 427,  219,  178,  127,  484,   33,    1,  427,   18,   51, 1461,   16,
         176,  177,   47,  159,  163,  127,   33,    1,   47,   11,   51, 1290,
         130,  557,  127,  269, 1324,  130,  303,   18,    1,   47,  171,  269,
         699, 1311,  163,  127,   18,   47,   73,  302,  830,  128,   54,  959,
          81,  130,  175,   18,    1,  284, 1748,  499,  127,  553,   39,  241,
         565,   85,  634,  661,   33,    1,  157,   11,   57,  281,   18,    1,
         219,  234,  633,  535,   33,    1,  479,   16,  140,   11,   57,  128,
         451,  446,   18,    1,  284,  127,  254,  423,   47,   11,   42, 1298,
         162,  212,  127,   33])
hi how are you today ? [SEP] hi . mike , what can i do for you ? [SEP] i ' m calling to ask you somewhere to work . [SEP] i have some bad news for you . i ’ ve heard the piano to me . [SEP] did anyone say you were a good mangary class ? [SEP] that ' s right . [SEP] how about tomorro

In [8]:
### We wrape the language model into a pytorch lightning module. This makes training convinient and adds some more high-level functions.
# But it also requires giving some extra information to the model such as the hyperparameters
from modules.pytorch_lightning.LightningLanguageModel import LightningLanguageModel
hparams = {
    "learning_rate": 0.01,
    "teacher_forcing": True,
    "freeze_language_ml": True
}

lightning_language_model = LightningLanguageModel(trained_language_model, my_tokenizer, hparams=hparams)


In [9]:
## But it gives us some nice options such as easy completion of dialogues

dialogues = ["Hi how are you? ", "what are you upto? [SEP]"]

print(lightning_language_model.complete_dialogues(dialogues, max_length=100))

["hi how are you ? [SEP] hello , may i have a look at this time of bed or canada ? [SEP] no , thank you very much . [SEP] i ' m afraid i could . do you have any other questions ? [SEP] yes , i ' m in , too , i haven ’ t found anything in our new computer and i have worked in the same department . [SEP] i ' m not really sure to see you ? [SEP] yes , we will . [SEP] what about your family ? [SEP] do", "what are you upto ? [SEP] i don ' t know . what about you ? [SEP] i had a bad headache . [SEP] what happened ? [SEP] i got it on the meeting . i can ’ t believe it must be a wonderful timeer . [SEP] well , you can ’ t go out for me . [SEP] what ' s that ? [SEP] i ' m sorry . i didn ' t know you were not busy at the beginning of vacation , and there are something interesting . [SEP] i"]
